In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.io import fits
import scipy.sparse
import scipy.linalg

r0 = 2.818e-13 # classical electron radius in cm
m_e = 9.10938e-28 # electron mass in gram
c = 2.998e10 # speed of light in cm/s
h = 6.6261e-27 # planck constant in cm^2 g s-1
eV_to_erg = 1.6022e-12
parsec_to_cm = 3.085677581e18 # cm per parsec

k = 1.3808e-16 # boltzmann constant in cm^2 g s^-2 K^-1
T_CMB = 2.725 # CMB temperature today in K

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_p = 1.67262193269e-24 # proton mass in g
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium mass fraction
f_H = 0.76 # hydrogen fraction
Y_He = 0.24

E_e = m_e * c * c
rho_crit = 3 * H_0 * H_0 / 8 / np.pi / G
e = 2.718281828459 # base of natural log

z = 2
z2 = (1 + z) / 3

In [2]:
with open('rate_10Mpc.pkl', 'rb') as f:
    rata_arr = pkl.load(f)
    
rate_arr = np.array(rata_arr)
rate_arr = rate_arr.reshape(400,400) # [i][j]: gamma[i] theta[j], rate_arr[-1] = nan
rate_trans = np.transpose(rate_arr) # [i][j]: theta[i] gamma[j], rate_trans[i][-1] = nan

gamma_e_arr = np.logspace(8, 14, 400) * eV_to_erg / m_e / c**2 # lab-frame Lorentz factor of the electron produced
theta_e_arr = np.logspace(-8,0, 400)

In [3]:
def compton_f(x):
    if x > 1e-2:
        return ((-24 - 144*x - 290*x**2 - 194*x**3 + 20*x**4 + 28*x**5) / 3 / x**2 / (1+2*x)**3 + (4+4*x-x**2) / x**3 * np.log(1+2*x))
    return 8/5*x**2 - 32/5*x**3 + 2248/105*x**4 - 1376/21*x**5 + 11840/63*x**6

def compton_FF(x):
    n = 1024
    y = np.linspace(x / (2 * n), x - x / (2 * n), n)
    result = 0.
    for i in range(n): result += compton_f(y[i]) * y[i]
    result *= x / n
    return result

In [4]:
jtable = np.loadtxt('EBL_KS18_Q20_z_ 2.0.txt')
nstep = np.shape(jtable)[0] - 1
jtable[:,0] = 2.99792458e18 / jtable[:,0] # convert to Hz
nu = (jtable[:-1,0] + jtable[1:,0]) / 2
dnu = (jtable[:-1,0] - jtable[1:,0])
JnuEBL = (jtable[:-1,1] + jtable[1:,1]) / 2

xx = h*nu/k/T_CMB
Bnu = 2 * h * nu**3 / c**2 / np.expm1(xx)
Jnu = JnuEBL + Bnu

/var/folders/k2/qxng019s7y17hp_kgkzy9b180000gn/T/ipykernel_1376/3569518853.py:9: RuntimeWarning: overflow encountered in expm1
  Bnu = 2 * h * nu**3 / c**2 / np.expm1(xx)


In [5]:
def sumsq(jp, j): # theta_rms from gamma_jp to gamma_j
    sumsq = 0
    for i in range(jp, j-1, -1):
        all_int = 0
        factors = Bnu / nu**3 * dnu
        for k in range(nstep):
            all_int += compton_FF(2 * gamma_e_arr[i] * h * nu[k] / m_e / c**2) * factors[k]
            
        tp2 = 135 * m_e * c**2 / 128 / np.pi**4 / gamma_e_arr[i]**3 * (m_e * c**2 / k / T_CMB)**4 * all_int
        sumi = tp2 / (m_e * c * gamma_e_arr[i])**2 * np.log(gamma_e_arr[1] / gamma_e_arr[0])
        sumsq += sumi
        
    return sumsq

In [6]:
print(sumsq(380, 330))

7.857315389265371e-92


In [14]:
print(prefix_sums[330] - prefix_sums[381])

7.857315389265371e-92


I using dynamic programming to calculate theta_rms more efficiently and checked the accurancy

In [7]:
import time
n = 400
time_start = time.time()

prefix_sums = [0 for _ in range(n)] # theta_rms from gamma_jp to gamma_j = np.sqrt(prefix[j] - prefix[jp+1])

for j in range(n-1, -1, -1):
    print(j, time.time()-time_start)
    all_int = 0
    factors = Bnu / nu**3 * dnu
    for k in range(nstep):
        all_int += compton_FF(2 * gamma_e_arr[j] * h * nu[k] / m_e / c**2) * factors[k]

    tp2 = 135 * m_e * c**2 / 128 / np.pi**4 / gamma_e_arr[j]**3 * (m_e * c**2 / k / T_CMB)**4 * all_int
    sumi = tp2 / (m_e * c * gamma_e_arr[j])**2 * np.log(gamma_e_arr[1] / gamma_e_arr[0])

    prefix_sums[j] = sumi

    if j + 1 < n:
        prefix_sums[j] += prefix_sums[j + 1]

399 0.00044989585876464844
398 3.711913824081421
397 7.3876588344573975
396 11.06358289718628
395 14.768094778060913
394 18.442911863327026
393 22.114091873168945
392 25.78519868850708
391 29.453852891921997
390 33.13551878929138
389 36.804327964782715
388 40.46848487854004
387 44.14194083213806
386 47.82068872451782
385 51.48216700553894
384 55.14633774757385
383 58.808090686798096
382 62.47346305847168
381 66.13383388519287
380 69.79072999954224
379 73.44461798667908
378 77.10198283195496
377 80.76833581924438
376 84.4578549861908
375 88.12700581550598
374 91.96363687515259
373 95.71191763877869
372 99.41548180580139
371 103.13138580322266
370 106.77127981185913
369 110.41877388954163
368 114.06196093559265
367 117.71318578720093
366 121.35891580581665
365 125.00027799606323
364 128.66904091835022
363 132.31096863746643
362 135.94156289100647
361 139.57073783874512
360 143.24253797531128
359 146.89651584625244
358 150.52657103538513
357 154.15890383720398
356 157.78965783119202
355 1

In [8]:
def calculate_solid_angles(theta):
    # theta_half[i] = theta[i-1/2]
    theta_half = (theta[1:] + theta[:-1]) / 2
    theta_half = np.concatenate([[theta[0]/2], theta_half, [3*theta[-1]/2 - theta[-2]/2]])

    Omega = 2 * np.pi * (theta_half[2:]**2 - theta_half[:-2]**2) / 2  # solid angles of annulus
    Omega = np.concatenate([Omega, [2 * np.pi * (theta[-1]**2 - theta_half[-2]**2)]])  # add the last circle

    return Omega, theta_half

In [9]:
Omega, theta_half = calculate_solid_angles(theta_e_arr)

In [10]:
def smoothed_Fourier_transform(f, tau = 1):
    # compute the Fourier transform
    F = np.fft.fft(f)
    
    # define the Gaussian window function
    n = len(f)
    t = np.arange(n)
    window = np.exp(-0.5 * ((t - n/2) / tau)**2)
    
    # apply the smoothing operation using the Gaussian window
    F1 = F*window
    # take the inverse Fourier Transform to obtain the smoothed Fourier Transform
    F1 = np.fft.ifft(F1)
    
    return F1

This is my thinking of building M matrix, but I'm not sure if it looks right as you say we don't need to calculate
f1 explicitly

1: $\sum_{i=0}^{x-1} \sum_j \Omega[i]M[i,j]f_1[j] = Y[x-1]$

2: $\sum_{i=0}^x \sum_j \Omega[i]M[i,j]f_1[j] = Y[x]$

2-1: $\sum_j \Omega[x]M[x,j]f_1[j] = Y[x] - Y[x-1]$

$\Omega[x]$ isn't depend on j, so can be taken out, then

$\sum_jM[x,j]f_1[j] = (Y[x]-Y[x-1])/\Omega[x]$

$\sum_jM[x,j]f_1[j] = dot(f_1, M[x].T) = dot(M[x], f_1.T)$

In [29]:
def build_M(f, theta, Omega, theta_half):
    M = np.zeros((400, 400))
    Y = np.zeros((400, 1)) # the right-hand-side of the sum for each imax
    
    f1 = smoothed_Fourier_transform(f)
    #print('f1', f1)
    
    for i in range(1, 399): # i = imax, imax is [1, 399)
        Y[i] = -2 * np.pi * theta_half[i] * (f1[i] - f1[i-1]) / (theta[i] - theta[i-1])
        
    for i in range(1, 399):
        rhs = (Y[i] - Y[i-1]) / Omega[i]
        mi = np.append((rhs / f1 / 399), 0)
        M[i] = mi.reshape(1, 400)
    
    #for j in range(N):
    #    sum_OM = 0
    #    for i in range(1, N):
    #        sum_OM += Omega[i] * M[i][j]
    #    M[0][j] = -sum_OM / Omega[0] # sum_i Omega[i]*M[i][j] = 0 for all j
    
    return M

In [33]:
def test_M(f, theta, Omega, theta_half):
    M = np.zeros((400, 400))
    Y = np.zeros((400, 1)) # the right-hand-side of the sum for each imax
    
    f1 = np.append(f, 0)
    
    for i in range(1, 399): # i = imax, imax is [1, 399)
        Y[i] = -2 * np.pi * theta_half[i] * (f1[i] - f1[i-1]) / (theta[i] - theta[i-1])
        
    for i in range(1, 399):
        rhs = (Y[i] - Y[i-1]) / Omega[i]
        M[i] = (rhs / f1 / 400).reshape(1, 400)
    
    return M

I try 2 different f array and it gives me different M matrix

In [36]:
M1 = test_M(rate_trans[0, :-1], theta_e_arr, Omega, theta_half)
M2 = test_M(rate_trans[1, :-1], theta_e_arr, Omega, theta_half)

print(M1[100], M2[100])

[-5.31749045e+14 -4.88716802e+14 -4.48838037e+14 -4.11751452e+14
 -3.77335904e+14 -3.45508529e+14 -3.16030752e+14 -2.88714784e+14
 -2.63521409e+14 -2.40304798e+14 -2.18851040e+14 -1.99115233e+14
 -1.81019835e+14 -1.64379880e+14 -1.49115183e+14 -1.35174684e+14
 -1.22431867e+14 -1.10776909e+14 -1.00166954e+14 -9.05216066e+13
 -8.17241773e+13 -7.37328910e+13 -6.65000153e+13 -5.99584438e+13
 -5.40530655e+13 -4.87351914e+13 -4.39350435e+13 -3.95883986e+13
 -3.56664265e+13 -3.21325893e+13 -2.89420017e+13 -2.60671244e+13
 -2.34820069e+13 -2.11524711e+13 -1.90525318e+13 -1.71642583e+13
 -1.54660883e+13 -1.39359380e+13 -1.25599857e+13 -1.13232240e+13
 -1.02076273e+13 -9.20164863e+12 -8.29641048e+12 -7.47999023e+12
 -6.74222150e+12 -6.07736217e+12 -5.47829872e+12 -4.93707631e+12
 -4.44887007e+12 -4.00927071e+12 -3.61259845e+12 -3.25441243e+12
 -2.93186441e+12 -2.64133366e+12 -2.37936785e+12 -2.14354789e+12
 -1.93141881e+12 -1.74040800e+12 -1.56851352e+12 -1.41403546e+12
 -1.27511184e+12 -1.15013

/var/folders/k2/qxng019s7y17hp_kgkzy9b180000gn/T/ipykernel_16130/382728207.py:12: RuntimeWarning: divide by zero encountered in true_divide
  M[i] = (rhs / f1 / 400).reshape(1, 400)


$n_{ij}(t) = 1/(\Gamma_{IC,0}\ \gamma_j^2) ∑_{j’=j}^{j’_{max}} ∆gamma_{j’} ∑_{i’} [exp(theta_{rms, j’->j}^2/4*M)]_{ii’} S_{i’j’}$

In [19]:
def get_nij(i, j, t): # nij = dN / (dV dgammae_j dOmegae_i)
    Gamma_IC0 = 1.1e-18 * z2**4
    gamma_max_inv = 1. / gamma_e_arr[j] - Gamma_IC0 * t
    if gamma_max_inv>1e-99:
        gamma_max = 1./gamma_max_inv
    else:
        gamma_max = 1e99
        
    nij = 0
    
    for jp in range(j, 399):
        if gamma_e_arr[jp] <= gamma_max:
            theta_rms = np.sqrt(prefix_sums[j] - prefix_sums[jp+1])
            M = build_M(rate_trans[i, :-1], theta_e_arr, Omega, theta_half)
            M_expm = scipy.linalg.expm(theta_rms**2/4 * M)
            sum_MS = np.dot(M[i], rate_trans[:, jp])
                
            sum_ij = (gamma_e_arr[jp] - gamma_e_arr[jp-1]) * sum_MS
            
            if gamma_e_arr[jp+1] > gamma_max:
                fraction = (gamma_max - gamma_e_arr[jp]) / (gamma_e_arr[jp+1] - gamma_e_arr[jp])
                sum_ij = sum_ij * fraction
                if i==0 and j%50==25: print(t, j, jp, fraction)
            nij += sum_ij
    nij = nij / (Gamma_IC0 * gamma_e_arr[j]**2)
    return nij

In [20]:
t_arr = [1e8, 5e8, 1e9, 5e9, 1e10, 5e10, 1e11, 5e11, 1e12, 5e12, 1e13]
t_num = len(t_arr)

In [ ]:
nij = np.zeros((t_num, 400, 400)) # t theta[i] gamma[j]
time_start = time.time()
for k in range(t_num):
    print('time', k, t_arr[k], time.time()-time_start)
    for i in range(400):
        for j in range(1, 400):
            nij[k][i][j-1] = get_nij(i, j, t_arr[k])